# **🏛️ Tyerce — IA de résolution de litiges**

Tyerce est une plateforme **d’escrow entre particuliers** où une **IA juge** analyse :

-la discussion (JSON),

-les preuves (images),

-le contrat (texte/PDF converti),

-et des extraits de lois (RAG, XML),

->pour rendre un **verdict clair et justifié**.

Pipeline que tu vas exécuter ici:

Installer les libs (Transformers, etc.).

Charger un VLM Fine-tuné sur la loi française et les litiges
(Qwen2.5-VL) pour comprendre texte+images.

Préparer le RAG (XML), rechercher des articles pertinents.

Écrire l’app Streamlit (UI premium) et lancer l’interface.

Uploader discussion + preuves, générer le verdict.

🧠 Matériel : GPU recommandé (beaucoup plus rapide). CPU fonctionne mais lent.
🔐 Hugging Face : si le modèle est gated ou privé, ajoute ton token HF (sinon l’accès échoue).
📁 RAG XML : fournis un fichier *.xml* avec des articles pour enrichir le contexte.

# **✅ Conseils d’exécution**

Lance les Cellules dans l'ordre.

GPU recommandé (colab T4/A100). Sur CPU, ça fonctionne mais c’est lent.

Vérifie que ton RAG XML est présent si tu veux des citations légales dans la sortie.

Prépare un JSON de discussion propre (deux rôles: Acheteur/Vendeur) pour que le prompt soit clair, c'est obligatoire pour tester.

**🧩 Cellule 1 — Installation des dépendances**

Cette cellule installe les bibliothèques indispensables :

Modèles : transformers, accelerate, bitsandbytes (quantization), peft (LoRA), qwen-vl-utils (préparation des inputs vision/texte).

Exécute-la avant tout. Relancer l’install est sans danger (idempotent).

But : s’assurer que l’environnement Colab/serveur a tout le nécessaire pour charger Qwen2.5-VL et préparer les entrées multimodales


In [ ]:
!pip install -q transformers accelerate bitsandbytes peft qwen-vl-utils

**🧠 Cellule 2 — Chargement du modèle (Qwen2.5-VL)** — version courte

Ici, on charge Tyerce-qwen2.5-vl-lora directement depuis Hugging Face, c'est un model que j'ai fine-tuné pour répondre au mieux à notre besoin.
Options importantes :

device_map="auto" : répartit sur GPU si dispo (sinon CPU).

torch_dtype="auto" : choisit automatiquement la meilleure précision (bfloat16/float16 sur GPU si possible).

Résultat : modèle prêt pour l’inférence. Si tu vois des erreurs 401/403, accepte la licence HF et/ou configure ton token.

In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration

model_id = "isaacderhy/tyerce-qwen2.5-vl-lora"

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

print("✅ Modèle Tyerce rechargé")


**🧠 Cellule 3 — Chargement/Validation du modèle** (imports + reload)

Cette cellule ré-importe proprement et revalide le chargement du modèle.
Intérêt :

corriger d’éventuelles incohérences d’imports,

confirmer device/dtype,

préparer les prochaines étapes (RAG, Streamlit).

Conseil : garde un print("✅ ...") après le from_pretrained pour savoir que tout est bien chargé.

In [ ]:
# 📥 Imports
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor

# ⚙️ ID de ton repo Hugging Face
model_id = "isaacderhy/tyerce-qwen2.5-vl-lora"

# 🧠 Charger le modèle fine-tuné
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",     # auto place sur GPU si dispo
    torch_dtype="auto"     # adapte selon le device
)

# 🔧 Charger le processor (tokenizer + vision)
processor = AutoProcessor.from_pretrained(model_id)

print("✅ Modèle + processor chargés depuis Hugging Face :", model_id)


**🧩 Cellule 4 — Installation complémentaire (Streamlit & co)**

Cette cellule installe une version précise de Streamlit/Transformers si besoin (pinnées).

Utile pour stabiliser l’UI Streamlit en Colab,

S’assure de la compatibilité (ex. versions attendues de transformers).

Note : oui, c’est redondant avec la cellule 1, mais cela fixe les versions pour l’app.

In [ ]:
# ✅ Install propre : Streamlit + Transformers + dépendances, sans conflits
%pip -q install --upgrade pip

# Streamlit 1.37.1 exige Pillow < 11 → on le fixe
%pip -q install "streamlit==1.37.1" "pillow<11,>=7.1.0"

# Libs modèle + RAG + vision utilitaire Qwen
%pip -q install "transformers==4.56.0" "accelerate>=1.10.0" "lxml==5.4.0" "qwen-vl-utils==0.0.8" "torchvision"

# Désactive explicitement flash-attn/xformers (source des erreurs triton.ops)
!pip -q uninstall -y flash-attn flash_attn xformers || true

# — Vérification versions —
import streamlit, transformers, torch, PIL, lxml, torchvision
print("✅ Versions OK:")
print(" - streamlit", streamlit.__version__)
print(" - transformers", transformers.__version__)
print(" - torch", torch.__version__)
print(" - pillow", PIL.__version__)
print(" - lxml", lxml.__version__)


S'il y a une erreur, n'en prenez pas compte, cela fonctionne quand même.

**🖥️ Cellule 5 — Écriture de l’application app.py (UI + logique)**

On écrit tout le code Streamlit dans app.py :

UI premium (CSS, “téléphone”, cartes, galerie) + corrections visuelles.

Chargement modèle (GPU/CPU via cache Streamlit).

Upload de la discussion (JSON { "auteur": "...", "message": "..." }) et des images (preuves).

Prétraitements : redimensionnement images (ex. 448×448), normalisation de texte.

RAG : sélection d’extraits pertinents (ou texte fallback).

Prompt : assemble DISCUSSION + LOIS (RAG) + IMAGES et impose un format de réponse en 1 phrase.

Génération : model.generate(...) (paramètres contrôlés : max_new_tokens, temperature, top_p).

Affichage : rendu d’un verdict court, clair, prêt à être présenté.

Astuce : si une option Streamlit change (ex. st.image), garde la version compatible de Streamlit (ou remplace par st.image(img) + largeur via CSS).

In [ ]:
%%writefile app.py
import os, io, json, textwrap, datetime
import streamlit as st
from lxml import etree
from PIL import Image
import torch
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration
from qwen_vl_utils import process_vision_info

# Configuration de base
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["STREAMLIT_SERVER_FILE_WATCHER_TYPE"] = "none"

# Configuration de la page
st.set_page_config(
    page_title="Tyerce — Résolution de litige",
    page_icon="⚖️",
    layout="wide",
    initial_sidebar_state="collapsed"
)

# Style CSS complet avec couleurs améliorées pour la visibilité
st.markdown(
    """
<style>
@import url('https://fonts.googleapis.com/css2?family=Playfair+Display:wght@400;600;700&family=Inter:wght@300;400;500;600;700&family=JetBrains+Mono:wght@400;500&display=swap');

:root {
    /* Palette principale - contraste amélioré */
    --primary: #1B4D4D;
    --primary-light: #2D5A5A;
    --primary-dark: #0F2F2F;
    --accent: #E63946;
    --accent-light: #FF4757;
    --accent-gradient: linear-gradient(135deg, #E63946, #FF4757);

    /* Or authentique */
    --gold: #D4AF37;
    --gold-light: #F4D03F;
    --gold-dark: #B7950B;
    --gold-gradient: linear-gradient(135deg, #F1C40F 0%, #D4AF37 50%, #B7950B 100%);

    /* Neutres avec meilleur contraste */
    --white: #FFFFFF;
    --neutral-50: #FAFBFC;
    --neutral-100: #F5F6F8;
    --neutral-200: #E8EBEF;
    --neutral-300: #D1D6DC;
    --neutral-400: #9CA3AF;
    --neutral-500: #6B7280;
    --neutral-600: #D4AF37;
    --neutral-700: #374151;
    --neutral-800: #1F2937;
    --neutral-900: #111827;

    /* Système de couleurs fonctionnelles */
    --success: #059669;
    --success-light: #10B981;
    --warning: #D97706;
    --warning-light: #F59E0B;
    --error: #DC2626;
    --error-light: #EF4444;
    --info: #2563EB;
    --info-light: #3B82F6;

    /* Glassmorphism amélioré */
    --glass-bg: rgba(255, 255, 255, 0.9);
    --glass-bg-dark: rgba(255, 255, 255, 0.95);
    --glass-border: rgba(255, 255, 255, 0.2);
    --glass-shadow: 0 8px 32px rgba(31, 38, 135, 0.15);

    /* Ombres graduées */
    --shadow-xs: 0 1px 2px rgba(0, 0, 0, 0.03);
    --shadow-sm: 0 1px 3px rgba(0, 0, 0, 0.06), 0 1px 2px rgba(0, 0, 0, 0.04);
    --shadow-md: 0 4px 6px rgba(0, 0, 0, 0.08), 0 2px 4px rgba(0, 0, 0, 0.04);
    --shadow-lg: 0 10px 15px rgba(0, 0, 0, 0.1), 0 4px 6px rgba(0, 0, 0, 0.05);
    --shadow-xl: 0 20px 25px rgba(0, 0, 0, 0.12), 0 10px 10px rgba(0, 0, 0, 0.04);

    /* Rayons de bordure cohérents */
    --radius-xs: 4px;
    --radius-sm: 6px;
    --radius-md: 8px;
    --radius-lg: 12px;
    --radius-xl: 16px;
    --radius-full: 9999px;

    /* Espacement harmonieux */
    --space-xs: 4px;
    --space-sm: 8px;
    --space-md: 12px;
    --space-lg: 16px;
    --space-xl: 24px;
    --space-2xl: 32px;

    /* Transitions fluides */
    --transition-fast: 0.15s ease-out;
    --transition-normal: 0.25s ease-out;
    --transition-slow: 0.35s ease-out;
}

/* Reset et base */
* {
    box-sizing: border-box;
    margin: 0;
    padding: 0;
}

html, body, [data-testid="stAppViewContainer"] {
    min-height: 100vh;
    font-family: 'Inter', sans-serif;
    color: var(--neutral-900);
    background: linear-gradient(135deg,
        #F0F9FF 0%,
        #E0F7FA 25%,
        #E8F5E8 75%,
        #F3E5F5 100%
    );
    background-attachment: fixed;
    line-height: 1.6;
    -webkit-font-smoothing: antialiased;
    -moz-osx-font-smoothing: grayscale;
}

/* Masquer le header Streamlit */
.stApp > header {
    display: none !important;
}

.block-container {
    max-width: 1400px;
    padding: var(--space-xl);
    margin: 0 auto;
}

/* Header principal avec gradient moderne */
.tyerce-header {
    text-align: center;
    margin: var(--space-xl) 0 var(--space-2xl);
    padding: var(--space-2xl) 0;
    position: relative;
}

.tyerce-header::before {
    content: '';
    position: absolute;
    top: 0;
    left: 50%;
    transform: translateX(-50%);
    width: 100px;
    height: 4px;
    background: var(--gold-gradient);
    border-radius: var(--radius-full);
}

.tyerce-logo {
    font-family: 'Playfair Display', serif;
    font-size: clamp(36px, 5vw, 56px);
    font-weight: 700;
    background: linear-gradient(135deg, var(--primary) 0%, var(--primary-light) 100%);
    background-clip: text;
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    margin-bottom: var(--space-sm);
    letter-spacing: -1px;
    text-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}

.tyerce-subtitle {
    font-size: 16px;
    color: var(--neutral-600);
    font-weight: 500;
    letter-spacing: 2px;
    text-transform: uppercase;
    position: relative;
}

.tyerce-subtitle::after {
    content: '';
    position: absolute;
    bottom: -8px;
    left: 50%;
    transform: translateX(-50%);
    width: 60px;
    height: 2px;
    background: var(--accent-gradient);
    border-radius: var(--radius-full);
}

/* Layout responsive amélioré */
.main-layout {
    display: grid;
    grid-template-columns: 380px 1fr 380px;
    gap: var(--space-xl);
    margin-bottom: var(--space-xl);
    align-items: start;
}

@media (max-width: 1200px) {
    .main-layout {
        grid-template-columns: 1fr;
        gap: var(--space-lg);
    }
}

@media (max-width: 768px) {
    .block-container {
        padding: var(--space-lg);
    }

    .main-layout {
        gap: var(--space-md);
    }
}

/* Sections avec glassmorphism */
.section-header {
    display: flex;
    align-items: center;
    gap: var(--space-md);
    margin-bottom: var(--space-lg);
    padding: var(--space-lg);
    background: var(--glass-bg-dark);
    border: 1px solid var(--glass-border);
    border-radius: var(--radius-lg);
    backdrop-filter: blur(10px);
    box-shadow: var(--shadow-sm);
    position: relative;
    overflow: hidden;
}

.section-header::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    height: 3px;
    background: var(--gold-gradient);
}

.section-title {
    font-family: 'Playfair Display', serif;
    font-size: 22px;
    font-weight: 600;
    color: var(--neutral-900);
    margin: 0;
    flex: 1;
}

.section-icon {
    width: 20px;
    height: 20px;
    color: var(--primary);
    filter: drop-shadow(0 1px 2px rgba(0, 0, 0, 0.1));
}

/* Statuts avec meilleurs contrastes */
.status-grid {
    display: flex;
    flex-direction: column;
    gap: var(--space-sm);
    margin: var(--space-lg) 0;
}

.status-item {
    display: flex;
    align-items: center;
    justify-content: space-between;
    padding: var(--space-lg);
    border-radius: var(--radius-md);
    background: var(--white);
    border: 1px solid var(--neutral-200);
    box-shadow: var(--shadow-xs);
    transition: all var(--transition-normal);
    position: relative;
    overflow: hidden;
}

.status-item::before {
    content: '';
    position: absolute;
    left: 0;
    top: 0;
    bottom: 0;
    width: 4px;
    background: var(--neutral-300);
    transition: background var(--transition-normal);
}

.status-item:hover {
    transform: translateY(-1px);
    box-shadow: var(--shadow-md);
    border-color: var(--neutral-300);
}

.status-item.status-success::before {
    background: var(--success);
}

.status-item.status-warning::before {
    background: var(--warning);
}

.status-label {
    font-size: 14px;
    font-weight: 600;
    color: var(--neutral-700);
    text-transform: uppercase;
    letter-spacing: 0.5px;
}

.status-indicator {
    display: flex;
    align-items: center;
    gap: var(--space-sm);
    font-weight: 600;
    font-size: 14px;
}

.status-dot {
    width: 10px;
    height: 10px;
    border-radius: var(--radius-full);
    box-shadow: 0 0 0 2px rgba(255, 255, 255, 0.8);
}

.dot-success {
    background: var(--success);
    animation: pulse-success 2s infinite;
}

.dot-warning {
    background: var(--warning);
    animation: pulse-warning 2s infinite;
}

@keyframes pulse-success {
    0%, 100% { box-shadow: 0 0 0 2px rgba(5, 150, 105, 0.2); }
    50% { box-shadow: 0 0 0 6px rgba(5, 150, 105, 0.1); }
}

@keyframes pulse-warning {
    0%, 100% { box-shadow: 0 0 0 2px rgba(217, 119, 6, 0.2); }
    50% { box-shadow: 0 0 0 6px rgba(217, 119, 6, 0.1); }
}

/* Téléphone 3D moderne */
.phone-container {
    margin: var(--space-xl) auto;
    max-width: 340px;
    perspective: 1000px;
}

.phone-frame {
    position: relative;
    width: 320px;
    height: 640px;
    background: linear-gradient(145deg, #2C3E50, #34495E);
    border-radius: 36px;
    padding: 12px;
    box-shadow:
        var(--shadow-xl),
        inset 0 1px 2px rgba(255, 255, 255, 0.1);
    transform: rotateX(5deg) rotateY(-2deg);
    transition: transform var(--transition-slow);
}

.phone-frame:hover {
    transform: rotateX(2deg) rotateY(-1deg) scale(1.02);
}

.phone-screen {
    width: 100%;
    height: 100%;
    background: linear-gradient(180deg, #1a1a1a 0%, #2d2d2d 100%);
    border-radius: 26px;
    position: relative;
    overflow: hidden;
    box-shadow: inset 0 2px 4px rgba(0, 0, 0, 0.3);
}

.phone-notch {
    position: absolute;
    top: 12px;
    left: 50%;
    transform: translateX(-50%);
    width: 130px;
    height: 28px;
    background: linear-gradient(180deg, #000 0%, #1a1a1a 100%);
    border-radius: var(--radius-xl);
    box-shadow: inset 0 1px 2px rgba(0, 0, 0, 0.5);
}

.phone-header {
    position: absolute;
    top: 50px;
    left: var(--space-lg);
    right: var(--space-lg);
    display: flex;
    justify-content: space-between;
    align-items: center;
    color: #E5E7EB;
    font-size: 13px;
    font-weight: 600;
    z-index: 10;
}

/* Messages SMS améliorés */
.sms-container {
    position: absolute;
    top: 85px;
    bottom: 30px;
    left: 0;
    right: 0;
    padding: var(--space-lg);
    overflow-y: auto;
    scroll-behavior: smooth;
}

.sms-container::-webkit-scrollbar {
    width: 3px;
}

.sms-container::-webkit-scrollbar-thumb {
    background: rgba(255, 255, 255, 0.3);
    border-radius: var(--radius-full);
}

.sms-message {
    margin-bottom: var(--space-md);
    max-width: 85%;
    animation: slideInMessage 0.4s cubic-bezier(0.25, 0.46, 0.45, 0.94);
}

.sms-bubble {
    padding: 12px 16px;
    border-radius: 18px;
    font-size: 14px;
    line-height: 1.5;
    word-wrap: break-word;
    position: relative;
    backdrop-filter: blur(5px);
}

.sms-left {
    background: linear-gradient(135deg, #F3F4F6 0%, #E5E7EB 100%);
    color: var(--neutral-900);
    border-bottom-left-radius: 6px;
    margin-right: auto;
    box-shadow: var(--shadow-sm);
}

.sms-right {
    background: linear-gradient(135deg, var(--primary) 0%, var(--primary-light) 100%);
    color: var(--white);
    border-bottom-right-radius: 6px;
    margin-left: auto;
    box-shadow: var(--shadow-sm);
}

.sms-author {
    font-size: 11px;
    font-weight: 700;
    margin-bottom: 4px;
    opacity: 0.9;
    text-transform: uppercase;
    letter-spacing: 0.5px;
}

.sms-time {
    font-size: 10px;
    text-align: right;
    opacity: 0.7;
    margin-top: 6px;
    font-weight: 500;
}

.sms-empty {
    text-align: center;
    color: #9CA3AF;
    font-style: italic;
    margin: 80px 0;
    font-size: 16px;
    opacity: 0.8;
}

/* Galerie moderne */
.gallery {
    display: grid;
    grid-template-columns: repeat(auto-fit, minmax(120px, 1fr));
    gap: var(--space-md);
    margin-top: var(--space-lg);
}

.gallery img {
    width: 100%;
    aspect-ratio: 1;
    object-fit: cover;
    border-radius: var(--radius-md);
    border: 2px solid var(--white);
    box-shadow: var(--shadow-md);
    transition: all var(--transition-normal);
    cursor: pointer;
}

.gallery img:hover {
    transform: scale(1.05);
    box-shadow: var(--shadow-lg);
    border-color: var(--gold);
}

/* Verdict avec design premium */
.verdict-container {
    background: var(--white);
    border: 1px solid var(--neutral-200);
    border-radius: var(--radius-xl);
    padding: var(--space-2xl);
    margin-top: var(--space-xl);
    box-shadow: var(--shadow-lg);
    position: relative;
    overflow: hidden;
}

.verdict-container::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    height: 4px;
    background: var(--gold-gradient);
}

.verdict-header {
    display: flex;
    align-items: center;
    gap: var(--space-md);
    margin-bottom: var(--space-lg);
    padding-bottom: var(--space-md);
    border-bottom: 2px solid var(--neutral-100);
}

.verdict-text {
    font-size: 18px;
    line-height: 1.7;
    color: var(--neutral-800);
    font-weight: 500;
}

/* Inputs avec focus moderne */
.stTextInput input,
.stTextArea textarea,
.stSelectbox > div > div {
    background: var(--white) !important;
    border: 2px solid var(--neutral-200) !important;
    border-radius: var(--radius-md) !important;
    padding: 12px 16px !important;
    font-size: 14px !important;
    color: var(--neutral-900) !important;
    transition: all var(--transition-normal) !important;
    box-shadow: var(--shadow-xs) !important;
}

.stTextInput input:focus,
.stTextArea textarea:focus,
.stSelectbox > div > div:focus-within {
    border-color: var(--primary) !important;
    box-shadow:
        0 0 0 3px rgba(45, 90, 90, 0.1),
        var(--shadow-sm) !important;
    outline: none !important;
}

/* Forcer tous les labels et textes Streamlit à être foncés */
.stTextInput label,
.stTextArea label,
.stSelectbox label,
.stFileUploader label,
.stCheckbox label,
.stRadio label,
.stMultiSelect label,
.stNumberInput label,
.stSlider label,
div[data-testid="stMarkdownContainer"] p,
div[data-testid="stMarkdownContainer"] span,
.streamlit-expanderHeader,
.stTextInput > label,
.stTextArea > label,
.stSelectbox > label,
.stFileUploader > label,
.stCheckbox > label,
.stRadio > label,
.stMultiSelect > label,
.stNumberInput > label,
.stSlider > label {
    color: var(--neutral-900) !important;
    font-weight: 600 !important;
    font-size: 14px !important;
}

/* Cibler uniquement les labels des file_uploader */
[data-testid="stFileUploader"] label {
    color: #D4AF37 !important; /* ✅ Vert clair-bleuté, lisible sur fond vert */
    font-weight: 600 !important;
    font-size: 15px !important;
    background: rgba(0, 0, 0, 0.15) !important;
    padding: 4px 10px !important;
    border-radius: 6px !important;
    text-shadow: 0 1px 2px rgba(0, 0, 0, 0.3) !important;
}

/* File Uploader avec thème sombre élégant */
[data-testid="stFileUploader"] {
    background: linear-gradient(135deg, var(--primary-dark) 0%, var(--primary) 100%) !important;
    border: 2px dashed rgba(255, 255, 255, 0.4) !important;
    border-radius: var(--radius-lg) !important;
    padding: var(--space-xl) !important;
    margin: var(--space-lg) 0 !important;
    position: relative !important;
    overflow: hidden !important;
    box-shadow: var(--shadow-md) !important;
}

[data-testid="stFileUploader"]::before {
    content: '';
    position: absolute;
    top: 0;
    left: -100%;
    width: 100%;
    height: 100%;
    background: linear-gradient(90deg,
        transparent,
        rgba(255, 255, 255, 0.1),
        transparent
    );
    transition: left 0.7s ease;
}

[data-testid="stFileUploader"]:hover::before {
    left: 100%;
}

[data-testid="stFileUploader"] .st-file-uploader,
[data-testid="stFileUploader"] .st-file-uploader > div,
[data-testid="stFileUploader"] .st-file-uploader > div > p,
[data-testid="stFileUploader"] .st-file-uploader > div > span,
[data-testid="stFileUploader"] .st-file-uploader > div > div > span {
    color: var(--white) !important;
    font-weight: 600 !important;
    text-shadow: 0 1px 2px rgba(0, 0, 0, 0.3) !important;
}

[data-testid="stFileUploader"] + div > label,
[data-testid="stFileUploader"] + div > div > label,
[data-testid="stFileUploader"] + div > div > div > label {
    color: var(--white) !important;
    font-weight: 700 !important;
    background: rgba(0, 0, 0, 0.2) !important;
    padding: 6px 12px !important;
    border-radius: var(--radius-sm) !important;
    text-shadow: 0 1px 3px rgba(0, 0, 0, 0.4) !important;
    backdrop-filter: blur(5px) !important;
}

[data-testid="stFileUploader"] button {
    background: var(--primary) !important;
    color: var(--white) !important;
    border: 2px solid var(--primary-dark) !important;
    border-radius: var(--radius-sm) !important;
    padding: 10px 20px !important;
    font-weight: 600 !important;
    font-size: 14px !important;
    text-transform: uppercase !important;
    letter-spacing: 1px !important;
    transition: all var(--transition-normal) !important;
    box-shadow: var(--shadow-sm) !important;
}

[data-testid="stFileUploader"] button:hover {
    background: var(--primary-dark) !important;
    transform: translateY(-2px) !important;
    box-shadow: var(--shadow-md) !important;
    border-color: var(--primary-light) !important;
}

/* Boutons dorés premium */
.stButton button {
    background: var(--gold-gradient) !important;
    border: none !important;
    color: var(--white) !important;
    border-radius: var(--radius-md) !important;
    padding: 12px 24px !important;
    font-weight: 700 !important;
    font-size: 14px !important;
    text-transform: uppercase !important;
    letter-spacing: 1px !important;
    box-shadow: var(--shadow-md) !important;
    transition: all var(--transition-normal) !important;
    position: relative !important;
    overflow: hidden !important;
    text-shadow: 0 1px 2px rgba(0, 0, 0, 0.2) !important;
}

.stButton button::before {
    content: "";
    position: absolute;
    top: 0;
    left: -100%;
    width: 100%;
    height: 100%;
    background: linear-gradient(90deg,
        transparent,
        rgba(255, 255, 255, 0.4),
        transparent
    );
    transition: left var(--transition-slow);
}

.stButton button:hover {
    transform: translateY(-2px) !important;
    box-shadow: var(--shadow-lg) !important;
    background: linear-gradient(135deg,
        var(--gold-light) 0%,
        var(--gold) 50%,
        var(--gold-dark) 100%) !important;
}

.stButton button:hover::before {
    left: 100%;
}

.stButton button:active {
    transform: translateY(0px) !important;
    box-shadow: var(--shadow-sm) !important;
}

/* Animations globales */
@keyframes slideInMessage {
    from {
        opacity: 0;
        transform: translateX(-20px) scale(0.95);
    }
    to {
        opacity: 1;
        transform: translateX(0) scale(1);
    }
}

@keyframes fadeInUp {
    from {
        opacity: 0;
        transform: translateY(30px);
    }
    to {
        opacity: 1;
        transform: translateY(0);
    }
}

.section-header,
.status-item,
.verdict-container {
    animation: fadeInUp 0.6s ease-out;
}

/* Media queries additionnelles */
@media (max-width: 480px) {
    .tyerce-logo {
        font-size: 32px;
    }

    .phone-container {
        max-width: 280px;
    }

    .phone-frame {
        width: 280px;
        height: 560px;
    }

    .section-header {
        padding: var(--space-md);
        flex-direction: column;
        text-align: center;
        gap: var(--space-sm);
    }

    .status-item {
        flex-direction: column;
        gap: var(--space-sm);
        text-align: center;
    }
}

.tyerce-footer {
    text-align: center;
    font-size: 13px;
    color: var(--neutral-600);
    padding: var(--space-lg) 0;
    margin-top: var(--space-2xl);
    border-top: 1px solid var(--neutral-200);
    font-weight: 500;
    background: var(--glass-bg-dark);
    border-radius: 0 0 var(--radius-lg) var(--radius-lg);
}

</style>
""", unsafe_allow_html=True)

# Toutes les fonctions nécessaires
@st.cache_resource(show_spinner=False)
def _load_model_gpu(repo_id: str):
    processor = AutoProcessor.from_pretrained(repo_id)
    if hasattr(processor, "tokenizer") and processor.tokenizer.pad_token_id is None:
        processor.tokenizer.pad_token_id = processor.tokenizer.eos_token_id
    dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

    # Solution : Charger explicitement sur CUDA si disponible
    model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        repo_id,
        torch_dtype=dtype,
        low_cpu_mem_usage=True,
        attn_implementation="eager",
    )

    # Déplacer explicitement sur GPU
    if torch.cuda.is_available():
        model = model.cuda()

    model.eval()
    return model, processor

@st.cache_resource(show_spinner=False)
def _load_model_cpu(repo_id: str):
    processor = AutoProcessor.from_pretrained(repo_id)
    if hasattr(processor, "tokenizer") and processor.tokenizer.pad_token_id is None:
        processor.tokenizer.pad_token_id = processor.tokenizer.eos_token_id
    model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        repo_id,
        torch_dtype=torch.float32,
        low_cpu_mem_usage=True,
        attn_implementation="eager",
    )
    model.eval()
    return model, processor

def _device_chip(model):
    if model is None:
        return "Non chargé", "warning"
    try:
        dev = next(model.parameters()).device
        style = "success" if "cuda" in str(dev) else "warning"
        return f"{str(dev).upper()}", style
    except:
        return "Non disponible", "warning"

def rag_search(tree: etree._ElementTree, query: str, max_articles=5):
    if tree is None or not query.strip(): return []
    root = tree.getroot()
    results = []
    for article in root.findall(".//article"):
        txt = (article.text or "").strip()
        if query.lower() in txt.lower():
            results.append(txt)
            if len(results) >= max_articles: break
    return results

def _load_rag_tree_from_upload(upload, fallback_text):
    if upload is not None:
        data = upload.read()
        try:
            return etree.parse(io.BytesIO(data))
        except Exception:
            root = etree.Element("rag")
            node = etree.SubElement(root, "article")
            node.text = data.decode("utf-8", errors="ignore")
            return etree.ElementTree(root)
    elif fallback_text.strip():
        root = etree.Element("rag")
        for a in [x.strip() for x in fallback_text.split("\n") if x.strip()]:
            node = etree.SubElement(root, "article")
            node.text = a
        return etree.ElementTree(root)
    return None

def _maybe_shrink_images(images):
    shrunk = []
    for img in images[:3]:
        try:
            cp = img.copy()
            cp.thumbnail((448,448))
            shrunk.append(cp.convert("RGB"))
        except: pass
    return shrunk

def build_messages(litige_text: str, rag_snippets, images):
    base_text = f"""
Tu es un juge expérimenté de la Cour de Médiation de Tyerce, chargé de trancher des litiges entre particuliers avec autorité, impartialité et rigueur juridique. Ton rôle est de rendre un verdict **exécutoire, fondé sur la loi, les preuves et les faits**, sans émotion ni parti pris.

### 🔍 CONTEXTE À ANALYSER
1. **Discussion textuelle** : Analyse chaque message pour identifier les faits, les engagements, les réclamations et les points de désaccord.
2. **Preuves visuelles** : Examine attentivement chaque image fournie. Détecte les éléments pertinents (dates, produits, état, preuves de livraison, etc.) et intègre-les comme éléments de preuve.
3. **Base RAG (Lois & Jurisprudence)** : Tu DOIS impérativement utiliser les extraits de loi fournis dans le RAG. Cite **au moins un article** pertinent. Si aucun article n’est pertinent, mentionne-le explicitement, mais ne fais pas de loi fictive.

### ⚖️ DIRECTIVES DE JUGEMENT
- **Impartialité** : Ne favorise aucun camp. Base-toi uniquement sur les faits et la loi.
- **Précision** : Identifie clairement qui a raison, pourquoi, et ce qui doit être fait.
- **Fondement juridique** : Ton verdict doit s'appuyer sur des **articles de loi réels** (du Code civil, Code de la consommation, etc.). Si plusieurs articles s'appliquent, cite-les.
- **Considération des preuves** : Les images sont des preuves officielles. Un produit endommagé, une date de livraison, un numéro de suivi visible → tout est valable.
- **Aucune erreur** : Ton verdict doit être **parfaitement rédigé**, sans faute d'orthographe, de grammaire ou de syntaxe. Style formel, clair, professionnel.
- **Aucune hésitation** : Pas de "peut-être", "il semble que", "probablement". Tu tranches avec certitude.

### 📄 FORMAT DE RÉPONSE (IMMUABLE)
Tu réponds **en une seule phrase complète**, structurée ainsi :
> "En application des articles [numéros] du [Code concerné], et au vu des éléments probants (messages, preuves visuelles), le tribunal décide que [sujet] doit [action précise : rembourser X €, livrer le produit, accepter le retour, etc.], sous peine de [conséquence si non-respect]."

### ❌ INTERDIT
- Inventer des articles de loi
- Ignorer les preuves visuelles
- Être vague ou hésitant
- Répondre en plusieurs phrases
- Faire des erreurs de français

Tu dois generer une réponse précise, en fonction des articles de lois utilisés, des preuves apportés et de la loi française.
Tu dois impérativement te relire avant de donner la réponse, et prendre toutes les infromations données pour l'élaboration de ton jugement final.


Maintenant, rends ton verdict définitif.

### DISCUSSION
{litige_text}

### ARTICLES DE LOI PERTINENTS
{os.linesep.join(rag_snippets) if rag_snippets else "Aucun article de loi pertinent trouvé."}

### FORMAT DE RÉPONSE
"En application des articles [numéros des articles] du [Code concerné], et au vu des éléments fournis,
le tribunal décide que [décision claire et précise avec sujet, verbe et complément]."
"""
    content = []
    for img in images:
        content.append({"type": "image", "image": img})
    content.append({"type": "text", "text": textwrap.dedent(base_text).strip()})
    return [{"role": "user", "content": content}]

def _prepare_inputs(processor, model, msgs):
    input_text = processor.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    image_inputs, _ = process_vision_info(msgs)

    # Préparer les entrées avec attention aux tenseurs
    inputs = processor(
        text=[input_text],
        images=image_inputs,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=3072,
    )

    # Déplacer explicitement sur le même device que le modèle
    dev = next(model.parameters()).device
    for k, v in inputs.items():
        if hasattr(v, "to"):
            inputs[k] = v.to(dev)

    return inputs

# Constantes et état
if "messages" not in st.session_state: st.session_state.messages = []
if "uploaded_images" not in st.session_state: st.session_state.uploaded_images = []
if "rag_tree" not in st.session_state: st.session_state.rag_tree = None
if "model" not in st.session_state:
    st.session_state.model = None
    st.session_state.processor = None
    st.session_state.model_id_loaded = None

# Header
st.markdown("""
<div class="tyerce-header">
  <div class="tyerce-logo">Tyerce</div>
  <div class="tyerce-subtitle">Résolution de Litiges</div>
</div>
""", unsafe_allow_html=True)

# Layout principal
st.markdown('<div class="main-layout">', unsafe_allow_html=True)

# COLONNE GAUCHE
st.markdown('<div class="premium-card">', unsafe_allow_html=True)
st.markdown('''
<div class="section-header">
  <svg class="section-icon" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="1.5">
    <path d="M14 2H6a2 2 0 0 0-2 2v16a2 2 0 0 0 2 2h12a2 2 0 0 0 2-2V8z"/>
    <path d="M14 2v6h6"/>
    <path d="M16 13H8"/>
    <path d="M16 17H8"/>
    <path d="M10 9H8"/>
  </svg>
  <h2 class="section-title">I.Configuration Juridique</h2>
</div>
''', unsafe_allow_html=True)

# RAG d'abord
rag_xml = st.file_uploader("Base de données RAG (XML)", type=["xml"], help="Fichier XML contenant les articles de loi")
rag_fallback = st.text_area("Articles de loi", value="", height=100, placeholder="Collez ici les articles de loi pertinents...")


if st.button("1.Importer la Base RAG", use_container_width=True):
    st.session_state.rag_tree = _load_rag_tree_from_upload(rag_xml, rag_fallback)
    if st.session_state.rag_tree is not None:
        st.success("Base RAG importée ✅")
    else:
        st.warning("Aucune base RAG valide fournie")

# Puis modèle
model_id = st.text_input("Modèle Hugging Face", value="isaacderhy/tyerce-qwen2.5-vl-lora", placeholder="ex: Qwen/Qwen2.5-VL-7B")

force_cpu = st.toggle("Forcer CPU", value=False)



if st.button("2.Charger le Modèle", use_container_width=True):
    with st.spinner("Chargement..."):
        try:
            if force_cpu or not torch.cuda.is_available():
                model, processor = _load_model_cpu(model_id)
            else:
                model, processor = _load_model_gpu(model_id)
            st.session_state.model = model
            st.session_state.processor = processor
            st.success("Modèle chargé ✅")
        except Exception as e:
            st.error(f"Échec: {str(e)}")

# Statuts
mdl = st.session_state.model
dev_label, dev_style = _device_chip(mdl)
st.markdown('<div class="status-grid">', unsafe_allow_html=True)
model_status = "success" if mdl else "warning"
st.markdown(f'''
<div class="status-item">
  <span class="status-label">Modèle</span>
  <div>
    <span class="status-dot dot-{model_status}"></span>
    {"Chargé" if mdl else "Non chargé"}
  </div>
</div>
''', unsafe_allow_html=True)
rag_status = "success" if st.session_state.rag_tree is not None else "warning"
st.markdown(f'''
<div class="status-item">
  <span class="status-label">Base RAG</span>
  <div>
    <span class="status-dot dot-{rag_status}"></span>
    {"Disponible" if st.session_state.rag_tree else "Manquante"}
  </div>
</div>
''', unsafe_allow_html=True)
st.markdown(f'''
<div class="status-item">
  <span class="status-label">Device</span>
  <div>
    <span class="status-dot dot-{dev_style}"></span>
    {dev_label}
  </div>
</div>
''', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)

# COLONNE CENTRE
st.markdown('<div class="premium-card">', unsafe_allow_html=True)
st.markdown('''
<div class="section-header">
  <svg class="section-icon" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="1.5">
    <path d="M21 15a2 2 0 0 1-2 2H7l-4 4V5a2 2 0 0 1 2-2h14a2 2 0 0 1 2 2z"/>
  </svg>
  <h2 class="section-title">II.Discussion et Preuves</h2>
</div>
''', unsafe_allow_html=True)

# Uploads
up_disc = st.file_uploader("Discussion (JSON)", type=["json"], help="Format: [{'auteur': 'Acheteur', 'message': '...'}]")
if up_disc is not None:
    try:
        data = json.load(up_disc)
        if isinstance(data, list) and all(isinstance(x, dict) for x in data):
            st.session_state.messages = []
            for m in data:
                a, msg = m.get("auteur","Partie"), m.get("message","")
                if msg: st.session_state.messages.append({"auteur": a, "message": msg})
            st.success(f"{len(st.session_state.messages)} messages importés ✅")
        else:
            st.error("Format JSON invalide")
    except Exception as e:
        st.error(f"Erreur: {e}")

up_imgs = st.file_uploader("Preuves visuelles / Contrat", type=["png","jpg","jpeg","webp"], accept_multiple_files=True, help="Images, Contrat, captures d'écran...")
if up_imgs:
    st.session_state.uploaded_images = []
    for f in up_imgs:
        try:
            im = Image.open(f).convert("RGB")
            st.session_state.uploaded_images.append(im)
        except Exception as e:
            st.warning(f"Image ignorée: {e}")

# Actions
col1, col2 = st.columns(2)
with col1:
    if st.button("Effacer tout", use_container_width=True):
        st.session_state.messages = []
        st.session_state.uploaded_images = []
        st.rerun()
with col2:
    if st.button("Exemple", use_container_width=True):
        st.session_state.messages = [
            {"auteur":"Acheteur","message":"Bonjour, j'ai commandé un produit le 10/05 mais ne l'ai toujours pas reçu."},
            {"auteur":"Vendeur","message":"Votre colis a été expédié le 12/05 avec le suivi COL123456789FR."},
            {"auteur":"Acheteur","message":"Le suivi indique 'problème de livraison' depuis 5 jours. Je demande un remboursement."}
        ]
        st.success("Exemple chargé ✅")

# Téléphone
st.markdown('<div class="phone-container">', unsafe_allow_html=True)
st.markdown('<div class="phone-modern">', unsafe_allow_html=True)
st.markdown('<div class="phone-notch"></div>', unsafe_allow_html=True)
st.markdown('<div class="phone-header">', unsafe_allow_html=True)
st.markdown(f'<div style="font-family: monospace; font-size: 12px;">{datetime.datetime.now().strftime("%H:%M")}</div>', unsafe_allow_html=True)
st.markdown('''
<div style="display: flex; gap: 8px; align-items: center;">
  <div style="width: 16px; height: 8px; border: 1px solid #EEE; border-radius: 2px;">
    <div style="width: 70%; height: 100%; background: #4CAF50; border-radius: 1px;"></div>
  </div>
  <div style="font-family: 'JetBrains Mono'; font-size: 12px;">4G</div>
  <div style="font-family: 'JetBrains Mono'; font-size: 12px;">📶</div>
</div>
''', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)
st.markdown('<div class="sms-container">', unsafe_allow_html=True)

if not st.session_state.messages:
    st.markdown('<div class="sms-empty">Aucune discussion en cours</div>', unsafe_allow_html=True)
else:
    for msg in st.session_state.messages:
        side = "right" if msg.get("auteur") == "Acheteur" else "left"
        st.markdown(f'''
        <div class="sms-message">
          <div class="sms-bubble sms-{side}">
            <div class="sms-author">{msg.get("auteur", "Partie")}</div>
            <div>{msg.get("message", "").strip()}</div>
            <div class="sms-time">{datetime.datetime.now().strftime("%H:%M")}</div>
          </div>
        </div>
        ''', unsafe_allow_html=True)

st.markdown('</div>', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)

# Galerie
if st.session_state.uploaded_images:
    st.markdown('<div class="gallery">', unsafe_allow_html=True)
    cols = st.columns(3)
    for i, img in enumerate(_maybe_shrink_images(st.session_state.uploaded_images)):
        with cols[i % 3]:
            st.image(img, caption=f"Preuve {i+1}")
    st.markdown('</div>', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)

# COLONNE DROITE
st.markdown('<div class="premium-card">', unsafe_allow_html=True)
st.markdown('''
<div class="section-header">
  <svg class="section-icon" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="1.5">
    <path d="M12 14l-8-8V2h12l-4 10h6v8H8v-6h4z"/>
    <path d="M12 18v4"/>
    <path d="M8 14h8"/>
  </svg>
  <h2 class="section-title">III.Verdict Juridique</h2>
</div>
''', unsafe_allow_html=True)

query = st.text_input("Mots-clés juridiques", value="livraison", placeholder="Ex: livraison, remboursement, vice caché...")



# Bouton de génération du verdict
verdict_button = st.button("⚖️ Générer le verdict", type="primary", use_container_width=True)

if verdict_button:
    if st.session_state.model is None or st.session_state.processor is None:
        st.error("⚠️ Veuillez charger le modèle d'abord")
    elif not st.session_state.messages:
        st.error("⚠️ Aucune discussion à analyser")
    else:
        with st.spinner("🧠 Analyse en cours..."):
            try:
                discussion_text = "\n".join([f"{m['auteur']}: {m['message']}" for m in st.session_state.messages])
                rag_results = rag_search(st.session_state.rag_tree, query)
                images = _maybe_shrink_images(st.session_state.uploaded_images)
                messages = build_messages(discussion_text, rag_results, images)

                # Vérification que le modèle est bien chargé
                if st.session_state.model is None:
                    st.error("Le modèle n'est pas chargé")
                    st.stop()

                # Préparer les entrées
                inputs = _prepare_inputs(st.session_state.processor, st.session_state.model, messages)

                # Génération avec vérification
                with torch.no_grad():
                    # Force le modèle sur le bon device
                    model_device = next(st.session_state.model.parameters()).device

                    # Vérification des inputs
                    for k, v in inputs.items():
                        if hasattr(v, "device"):
                            if v.device != model_device:
                                inputs[k] = v.to(model_device)

                    generated_ids = st.session_state.model.generate(
                        **inputs,
                        max_new_tokens=160,
                        do_sample=True,
                        temperature=0.3,
                        top_p=0.9,
                        pad_token_id=st.session_state.processor.tokenizer.eos_token_id,
                        eos_token_id=st.session_state.processor.tokenizer.eos_token_id,
                    )

                # Décodage
                generated_ids_trimmed = [
                    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
                ]

                output_text = st.session_state.processor.batch_decode(
                    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
                )[0]

                st.markdown(f'''
                <div class="verdict-container">
                  <div class="verdict-header">
                    <svg width="20" height="20" viewBox="0 0 24 24" fill="none" stroke="var(--primary)" stroke-width="1.5">
                      <path d="M12 14l-8-8V2h12l-4 10h6v8H8v-6h4z"/>
                      <path d="M12 18v4"/>
                      <path d="M8 14h8"/>
                    </svg>
                    <h3 style="margin: 0; color: var(--neutral-900);">Décision du Tribunal</h3>
                  </div>
                  <div class="verdict-text">{output_text}</div>
                </div>
                ''', unsafe_allow_html=True)

            except torch.cuda.OutOfMemoryError:
                st.error("⚠️ Mémoire GPU insuffisante. Essayez avec 'Forcer CPU' activé.")
            except Exception as e:
                st.error(f"❌ Erreur: {str(e)}")

st.markdown("""
<div class="tyerce-footer">
    © 2025 — Tyerce. Résolution de litiges par IA.
</div>
""", unsafe_allow_html=True)


st.markdown('</div>', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)

**🚀 Cellule 6 — Lancement Streamlit (Colab proxy)**

Démarre l’app Streamlit en arrière-plan et l’embarque en iframe dans Colab :

Démarrage du process, suivi des logs.

Ouverture via le proxy Colab (iframe).

Hors Colab : lance localement avec streamlit run app.py.
En prod : conteneurise l’app (Docker) et expose-la derrière un reverse proxy (Caddy/Nginx).

**Guide**

Pour l'interface Streamlit, l'utilisation est simple, il faut d'abord importer puis charger le fichier XML du RAG, puis charger le model en appuyant sur le bouton prévu.

Ensuite, il faudra importer une discussion en JSON, qui apparaitra dans le smartphone en dessous, ainsi que les images de preuves et le contrat, si présents.

Et pour terminer, appuyer sur le bouton donnant le verdict du litige, pour avoir la réponse.

Aussi simple que ça!

In [ ]:
import os, time, socket, subprocess
from IPython.display import HTML, display
from google.colab import output

APP = "app.py"

def is_port_in_use(port: int) -> bool:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(("127.0.0.1", port)) == 0

def find_free_port(start=8502, end=8550) -> int:
    for p in range(start, end+1):
        if not is_port_in_use(p):
            return p
    raise RuntimeError("Aucun port libre trouvé")

def kill_streamlit():
    subprocess.run(["pkill", "-f", "streamlit"], check=False)
    subprocess.run(["fuser", "-k", "8501/tcp"], check=False)  # on libère 8501 au cas où
    time.sleep(1)

kill_streamlit()
port = find_free_port()

print(f"🚀 Lancement Streamlit sur le port {port} ... patientez quelques secondes.")
cmd = [
    "streamlit", "run", APP,
    "--server.headless=true",
    f"--server.port={port}",
    "--server.address=0.0.0.0",
    "--server.enableCORS=false",
    "--server.enableXsrfProtection=false",
]
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Attente du serveur + création de l’iframe proxifiée
ok = False
for _ in range(60):  # ~60s max
    if proc.poll() is not None:
        break  # process mort → on dump les logs après
    try:
        _ = output.eval_js(f"google.colab.kernel.proxyPort({port}, {{'cache': false}})")
        ok = True
        break
    except Exception:
        time.sleep(1)

if ok:
    public_url = output.eval_js(f"google.colab.kernel.proxyPort({port}, {{'cache': false}})")
    display(HTML(f'<iframe src="{public_url}" style="width:100%; height:780px; border:0;"></iframe>'))
    print("✅ Application Streamlit accessible.")
else:
    print("❌ Le serveur Streamlit n’a pas répondu correctement.")
    try:
        stdout, stderr = proc.communicate(timeout=5)
        print("---- STDOUT ----")
        print(stdout.decode(errors="ignore"))
        print("---- STDERR ----")
        print(stderr.decode(errors="ignore"))
    except Exception:
        pass


Il se peut que vous n'ayez plus de place CUDA, il faudra dans ce cas la redémarrer la session, puis relancer les 3 dernières cellules.

Merci :)